In [3]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

### Load datasets

In [4]:
train_x_orig,train_y,test_x_orig,test_y,classes=load_dataset()

In [7]:
print(f'The shape of train_x_orig is {train_x_orig.shape}')
print(f'The shape of train_y is {train_y.shape}')
print(f'The shape of test_x is {test_x_orig.shape}')
print(f'The shape of test_y is {test_y.shape}')

The shape of train_x_orig is (209, 64, 64, 3)
The shape of train_y is (1, 209)
The shape of test_x is (50, 64, 64, 3)
The shape of test_y is (1, 50)


### Preprocess data

train_x_orig and test_x_orig needs to be **reshaped and standardized.**

In [15]:
train_flatten_x=train_x_orig.reshape(train_x_orig.shape[0],-1).T
test_flatten_x=test_x_orig.reshape(test_x_orig.shape[0],-1).T

To standardize, we divide by 255 because each image's RGB channels is 8 bits, so they are limited to 256.  \
We divide by 255 because 0 is included. By dividing with 255, all the values will be now between 0-1

In [18]:
train_x=train_flatten_x/255
test_x=test_flatten_x/255

### Define Sigmoid and Propagate Function for Logistic Model

In [150]:
def initialize_with_zeros(dim):
    b=0
    w=np.zeros((dim,1))
    return w,b

In [152]:
initialize_with_zeros(10)

(array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 0)

In [40]:
def sigmoid(x):
    s=1/(1+np.exp(-x))
    return s

In [207]:
def propagate(w,b,x,y):
    
    m=x.shape[1]
    z=np.dot(w.T,x)+b
    A=sigmoid(z)
    cost=-1/m * np.sum((y*np.log(A)+(1-y)*np.log(1-A)))
    dw = 1/m*(np.dot(x, ((A-y).T)))
    db = 1/m*(np.sum(A-y))
    grads={'dw':dw,'db':db}
    cost=np.squeeze(cost)
    return grads,cost
    

In [200]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])

In [201]:
def optimize(w,b,x,y,num_iter,learning_rate,print_cost=False):
    
    costs=[]
    
    for i in range(num_iter):
        
        grads,cost=propagate(w,b,x,y)
        
        dw=grads['dw']
        db=grads['db']
        
        w=w-learning_rate*dw
        db=b-learning_rate*db
        
        if i%100:
            costs.append(cost)
            
        if print_cost and i%100:
            print(f'The number of iteration and cost is {i,cost}')
            
    params={'w':w,'b':b}
    grads={'dw':dw,'db':db}
    
    return params,grads

In [202]:
def predict(x,w,b):
    
    z=np.dot(w.T,x)+b
    A=sigmoid(z)
    y_preds=[]
    
    for i in range(len(A.T)):
        if A[0][i]>0.5:
            y_preds.append(1) 
        else:
            y_preds.append(0)
    return np.squeeze(y_preds)

In [203]:
params,grads=optimize(w,b,X,Y,1000,0.01,print_cost=False)
grads

{'dw': array([[ 0.05801754],
        [-0.02506262]]),
 'db': 2.0005591559485416}

In [209]:
def logit_model(x_train, y_train, x_test, y_test , num_iter, learning_rate, print_cost=False):
    
    w,b=initialize_with_zeros(x_train.shape[0])
    params,grads=optimize(w,b, x_train,y_train,num_iter, learning_rate, print_cost)
   
    w=params['w']
    b=params['b']
    
    
    y_preds_x_train=predict(x_train,w,b)
    y_preds_x_test=predict(x_test,w,b)
   
    
    print(f'train_accuracy is {100-np.mean(np.abs(y_train-y_preds_x_train))}')
    print(f'test_accuracy is {100-np.mean(np.abs(y_test-y_preds_x_test))}')
    
    d={'learning_rate':learning_rate,
      'Y_prediction_train':y_preds_x_train,
      'Y_prediction_test':y_preds_x_test,
      'w':w,
      'b':b,
      'num_iteration':num_iter}
    
    return d

In [212]:
logit_model(train_x,train_y,test_x,test_y,1000,0.01,print_cost=True)

The number of iteration and cost is (1, 1.1197385607924608)
The number of iteration and cost is (2, 2.111124087054405)
The number of iteration and cost is (3, 3.197923681282844)
The number of iteration and cost is (4, 0.809641153837047)
The number of iteration and cost is (5, 1.88510111280681)
The number of iteration and cost is (6, 3.1862401589724665)
The number of iteration and cost is (7, 0.7945129772713704)
The number of iteration and cost is (8, 1.868196177901081)
The number of iteration and cost is (9, 3.1494498493504324)
The number of iteration and cost is (10, 0.7644136395028149)
The number of iteration and cost is (11, 1.799686612578311)
The number of iteration and cost is (12, 3.1110708493969845)
The number of iteration and cost is (13, 0.7351489560649106)
The number of iteration and cost is (14, 1.718897915095049)
The number of iteration and cost is (15, 3.067736964219075)
The number of iteration and cost is (16, 0.7046541843677593)
The number of iteration and cost is (17, 1

{'learning_rate': 0.01,
 'Y_prediction_train': array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 'Y_prediction_test': array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0]),
 'w': array([[ 0.01164712],
        [-0.03060873